<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания 


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

[ваш текст]

#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) исользуйтие в проекте коллекции, делегаты, события.


<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [1]:
using System;
using System.Collections.Generic;

#region Интерфейсы
public interface IPrintable
{
    void PrintSummary();
}

public interface IDiscountable
{
    void ApplyDiscount(decimal percent);
    void RemoveDiscount();
}

public interface IVIP
{
    bool IsVIP { get; set; }
    void GrantVIPAccess();
}

public interface ITicketProcessor
{
    void ProcessTicket(Ticket ticket);
    void Cancel(Ticket ticket);
}
#endregion

#region Делегаты и события

public delegate void TicketEventHandler(string message);
#endregion

#region Базовый класс Ticket
public class Ticket : IPrintable, IDiscountable
{

    public int TicketId { get; set; }
    public int RowNumber { get; set; }
    public int SeatNumber { get; set; }
    public decimal Price { get; set; }
    public DateTime EventDate { get; set; }
    public bool IsReserved { get; private set; }
    public Customer Owner { get; private set; }


    public string EventOrganizer { get; set; }
    public DateTime SaleDate { get; set; }
    public bool IsElectronic { get; set; }
    public string QRCode { get; private set; }
    public string EventName { get; set; }
    public string Location { get; set; }
    public string Category { get; set; }
    public bool HasDiscount { get; private set; }


    public List<string> ActionHistory { get; private set; }


    public event TicketEventHandler OnTicketChanged;

    public Ticket(int id, int row, int seat, decimal price, DateTime date, string eventName, string location, string category)
    {
        TicketId = id;
        RowNumber = row;
        SeatNumber = seat;
        Price = price;
        EventDate = date;
        EventName = eventName;
        Location = location;
        Category = category;

        SaleDate = DateTime.Now;
        IsElectronic = true;
        QRCode = "Не создан";
        EventOrganizer = "Не указан";

        ActionHistory = new List<string>();
    }

    public virtual void PrintTicketInfo()
    {
        Console.WriteLine($"Билет #{TicketId}: {EventName} ({Category}) — {Location}");
        Console.WriteLine($"Ряд {RowNumber}, место {SeatNumber}, цена {Price}₽, дата {EventDate:d}");
        Console.WriteLine($"Электронный: {IsElectronic}, QR: {QRCode}");
    }

    public virtual void PrintSummary()
    {
        Console.WriteLine($"[Сводка] Билет #{TicketId}: {EventName} — {Price}₽");
    }

    public void GenerateQRCode()
    {
        QRCode = Guid.NewGuid().ToString().Substring(0, 8);
        LogAction($"QR-код создан: {QRCode}");
        OnTicketChanged?.Invoke($"[Событие] QR для билета #{TicketId} создан.");
    }

    public void SendToEmail()
    {
        if (Owner != null)
            Console.WriteLine($"Билет #{TicketId} отправлен на e-mail: {Owner.Email}");
    }

    public void PrintReceipt()
    {
        Console.WriteLine($"Чек: покупатель {Owner?.Name}, сумма {Price}₽, дата продажи: {SaleDate:d}");
    }

    public virtual void ReserveSeat(Customer customer)
    {
        Owner = customer;
        IsReserved = true;
        LogAction($"Место {SeatNumber} зарезервировано для {customer.Name}");
        OnTicketChanged?.Invoke($"[Событие] Билет #{TicketId} зарезервирован для {customer.Name}");
    }

    public void CancelReservation()
    {
        if (IsReserved)
        {
            LogAction($"Резервация билета #{TicketId} отменена ({Owner?.Name})");
            IsReserved = false;
            Owner = null;
            OnTicketChanged?.Invoke($"[Событие] Билет #{TicketId} отменён.");
        }
    }

    public void ApplyDiscount(decimal percent)
    {
        Price -= Price * (percent / 100);
        HasDiscount = true;
        LogAction($"Скидка {percent}% применена");
        OnTicketChanged?.Invoke($"[Событие] Скидка применена ({percent}%)");
    }

    public void RemoveDiscount()
    {
        HasDiscount = false;
        LogAction("Скидка удалена");
    }


    protected void LogAction(string message)
    {
        ActionHistory.Add($"{DateTime.Now:T}: {message}");
    }


    public void PrintHistory()
    {
        Console.WriteLine($"\nИстория действий по билету #{TicketId}:");
        foreach (var h in ActionHistory)
            Console.WriteLine($" - {h}");
    }
}
#endregion

#region Производные классы
public class ConcertTicket : Ticket, IVIP
{
    public string ArtistName { get; set; }
    public string Genre { get; set; }
    public bool IsVIP { get; set; }

    // 🆕 Новые атрибуты
    public string DressCode { get; set; }
    public bool BackstageAccess { get; private set; }
    public List<string> SongList { get; private set; }

    public ConcertTicket(int id, int row, int seat, string artist, string venue, string genre, decimal price, DateTime date)
        : base(id, row, seat, price, date, artist, venue, "Концерт")
    {
        ArtistName = artist;
        Genre = genre;
        DressCode = "Smart Casual";
        SongList = new List<string> { "Intro", "Main Hit", "Encore" };
        BackstageAccess = false;
        IsVIP = false;
    }

    public void GrantVIPAccess()
    {
        if (!IsVIP)
        {
            IsVIP = true;
            BackstageAccess = true;
            Price *= 1.5m;
            Console.WriteLine($"VIP-доступ + Backstage активирован!");
        }
    }

    public void ShowDressCode() => Console.WriteLine($"Дресс-код: {DressCode}");

    public void BackstageEntry()
    {
        if (BackstageAccess)
            Console.WriteLine($"{Owner?.Name} входит за сцену!");
        else
            Console.WriteLine("Доступ к бекстейджу запрещён.");
    }


    public void ShowSetList()
    {
        Console.WriteLine($"Сет-лист {ArtistName}:");
        foreach (var song in SongList)
            Console.WriteLine($" - {song}");
    }

    public override void PrintSummary()
    {
        Console.WriteLine($"[Концерт] {ArtistName}, VIP: {IsVIP}, цена: {Price}₽");
    }
}

public class PlayTicket : Ticket
{
    public bool HasSubtitles { get; set; }
    public string TheaterHall { get; set; }

    public string Director { get; set; }
    public int Acts { get; set; }
    public Dictionary<string, string> Cast { get; set; }

    public PlayTicket(int id, int row, int seat, string title, string director, int acts, decimal price, DateTime date)
        : base(id, row, seat, price, date, title, "Театр", "Пьеса")
    {
        HasSubtitles = false;
        TheaterHall = "Главный зал";
        Director = director;
        Acts = acts;
        Cast = new Dictionary<string, string>
        {
            {"Гамлет", "Иван Петров"},
            {"Офелия", "Мария Смирнова"}
        };
    }

    public void ShowHall() => Console.WriteLine($"Постановка в зале: {TheaterHall}");
    public void SubtitleInfo() => Console.WriteLine(HasSubtitles ? "С субтитрами" : "Без субтитров");


    public void ShowCast()
    {
        Console.WriteLine($"Актёрский состав пьесы {EventName}:");
        foreach (var kv in Cast)
            Console.WriteLine($"{kv.Key} — {kv.Value}");
    }

    public override void PrintSummary()
    {
        Console.WriteLine($"[Пьеса] {EventName}, зал: {TheaterHall}, цена: {Price}₽");
    }
}
#endregion

#region Сервис и обработчик
public class TicketService
{
    private readonly ITicketProcessor _processor;

    public TicketService(ITicketProcessor processor)
    {
        _processor = processor;
    }

    public void Handle(Ticket t)
    {
        _processor.ProcessTicket(t);
    }

    public void Cancel(Ticket t)
    {
        _processor.Cancel(t);
    }
}

public class DefaultProcessor : ITicketProcessor
{
    public void ProcessTicket(Ticket ticket)
    {
        ticket.GenerateQRCode();
        ticket.PrintReceipt();
    }

    public void Cancel(Ticket ticket)
    {
        ticket.CancelReservation();
        Console.WriteLine("Процесс отмены завершён.");
    }
}
#endregion

#region Customer
public class Customer
{
    public string Name { get; set; }
    public string Email { get; set; }
    public List<Ticket> MyTickets { get; private set; }


    public event Action<string> OnTicketBought;

    public Customer(string name, string email)
    {
        Name = name;
        Email = email;
        MyTickets = new List<Ticket>();
    }

    public void BuyTicket(Ticket ticket)
    {
        ticket.ReserveSeat(this);
        MyTickets.Add(ticket);
        OnTicketBought?.Invoke($"Пользователь {Name} купил билет #{ticket.TicketId}");
    }

    public void PrintMyTickets()
    {
        Console.WriteLine($"\nБилеты пользователя {Name}:");
        foreach (var t in MyTickets)
            t.PrintSummary();
    }
}
#endregion

#region Демонстрация

        var concert = new ConcertTicket(1, 5, 12, "КняZZ", "ГлавClub", "Рок", 2500, new DateTime(2025, 10, 30));
        var play = new PlayTicket(2, 3, 7, "Гамлет", "Иванов", 3, 1800, new DateTime(2025, 11, 5));

        var customer = new Customer("Иван", "ivan@mail.com");
        customer.OnTicketBought += (msg) => Console.WriteLine($"[Событие покупателя] {msg}");

        concert.OnTicketChanged += (msg) => Console.WriteLine(msg);
        play.OnTicketChanged += (msg) => Console.WriteLine(msg);

        customer.BuyTicket(concert);
        ITicketProcessor processor = new DefaultProcessor();
        TicketService service = new TicketService(processor);

        service.Handle(concert);

        concert.ShowDressCode();
        concert.ShowSetList();
        concert.GrantVIPAccess();
        concert.BackstageEntry();

        Console.WriteLine();
        customer.BuyTicket(play);
        play.ShowHall();
        play.ShowCast();

        Console.WriteLine();
        customer.PrintMyTickets();

        concert.PrintHistory();

#endregion


[Событие] Билет #1 зарезервирован для Иван
[Событие покупателя] Пользователь Иван купил билет #1
[Событие] QR для билета #1 создан.
Чек: покупатель Иван, сумма 2500₽, дата продажи: 12.11.2025
Дресс-код: Smart Casual
Сет-лист КняZZ:
 - Intro
 - Main Hit
 - Encore
VIP-доступ + Backstage активирован!
Иван входит за сцену!

[Событие] Билет #2 зарезервирован для Иван
[Событие покупателя] Пользователь Иван купил билет #2
Постановка в зале: Главный зал
Актёрский состав пьесы Гамлет:
Гамлет — Иван Петров
Офелия — Мария Смирнова


Билеты пользователя Иван:
[Концерт] КняZZ, VIP: True, цена: 3750,0₽
[Пьеса] Гамлет, зал: Главный зал, цена: 1800₽

История действий по билету #1:
 - 17:36:58: Место 12 зарезервировано для Иван
 - 17:36:58: QR-код создан: 87ff4abf
